# Recommended: noVNC + code-server with cloudflared (Colab)

This notebook sets up an in-container Xfce desktop (Xvfb + x11vnc + noVNC), code-server (VS Code), and exposes both services publicly using cloudflared (not ngrok). This is the recommended approach for Colab because Chrome Remote Desktop (CRD) is fragile in ephemeral containers.

Security reminders:
- Change the default passwords before exposing publicly.
- Colab sessions are ephemeral; background processes will be terminated when the session ends.


In [ ]:
%%bash
set -e
echo "Installing desktop components, code-server, noVNC/websockify, and cloudflared (may take a few minutes)..."
apt-get update -qq
DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
  xfce4 xfce4-terminal x11vnc xvfb wget git python3-pip ca-certificates curl gnupg2 unzip \
  >/dev/null

# Install noVNC and websockify
rm -rf /opt/noVNC /opt/websockify
git clone https://github.com/novnc/noVNC.git /opt/noVNC
git clone https://github.com/novnc/websockify.git /opt/websockify

# Install cloudflared (latest release binary)
wget -q -O /usr/local/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
chmod +x /usr/local/bin/cloudflared

# Install code-server (VS Code in browser)
curl -fsSL https://code-server.dev/install.sh | sh

echo "Install finished. Proceed to start services in the next cell."


In [ ]:
%%bash
set -e
echo "Starting Xvfb + Xfce + x11vnc + noVNC + code-server"

# Default password; change before exposing publicly (or set env var PASSWORD in the notebook runtime)
PASSWORD=${PASSWORD:-"colab"}

export DISPLAY=:1
Xvfb $DISPLAY -screen 0 1280x800x24 >/tmp/xvfb.log 2>&1 &
sleep 1
nohup startxfce4 >/tmp/xfce.log 2>&1 &
sleep 2

x11vnc -display $DISPLAY -nopw -forever -shared -rfbport 5901 >/tmp/x11vnc.log 2>&1 &
sleep 1

# Start websockify (noVNC web UI) on 6080
/opt/websockify/run 6080 localhost:5901 --web=/opt/noVNC >/tmp/novnc.log 2>&1 &
sleep 1

mkdir -p ~/.config/code-server
cat > ~/.config/code-server/config.yaml <<EOF
bind-addr: 127.0.0.1:8080
auth: password
password: $PASSWORD
cert: false
EOF

nohup code-server --bind-addr 127.0.0.1:8080 --auth password >/tmp/code-server.log 2>&1 &

echo "Services started. code-server on 127.0.0.1:8080 (password: $PASSWORD). noVNC available on 127.0.0.1:6080."


In [ ]:
%%bash
set -e
echo "Starting cloudflared tunnels for code-server (8080) and noVNC (6080). Logs will show public URLs when available."
nohup cloudflared tunnel --url http://localhost:8080 > /tmp/cloudflared_8080.log 2>&1 &
nohup cloudflared tunnel --url http://localhost:6080 > /tmp/cloudflared_6080.log 2>&1 &
sleep 5
echo "-- cloudflared log (8080) --"
tail -n +1 /tmp/cloudflared_8080.log || true
echo "\n-- cloudflared log (6080) --"
tail -n +1 /tmp/cloudflared_6080.log || true

echo "If you see lines that include a .trycloudflare.com URL, those are the public endpoints. If not, give it a few seconds and re-run the tail command."


## How to use

1. Run the cells sequentially.
2. After the final cell completes, inspect the cloudflared log outputs for lines containing a URL like `https://<something>.trycloudflare.com` — those are your public endpoints.
3. Open the code-server URL and sign in with the password you set (env var `PASSWORD`, default `colab`).
4. Open the noVNC URL to access the Xfce desktop in your browser.

If you'd like, I can adjust the notebook to:
- Generate a random password at runtime and print it securely,
- Expose only code-server and keep VNC local (to avoid exposing both), or
- Produce a single downloadable .ipynb file packaged for you to upload to Colab.

Which tweak would you prefer?